
**You will have to insert the dataset "Meta-Kaggle" to run this...It gets updated every few hours or so.**

Dataset Link - https://www.kaggle.com/datasets/kaggle/meta-kaggle

Maintained by - Kaggle

In [ ]:
import os
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
episodeagents_df = pd.DataFrame()
for i, chunk in enumerate(pd.read_csv("../input/meta-kaggle/EpisodeAgents.csv", usecols=['EpisodeId','Reward','SubmissionId','InitialScore','UpdatedScore'], chunksize=1e6)):
    if i>66: episodeagents_df = pd.concat([episodeagents_df,chunk], ignore_index=True)       
episodeagents_df.fillna(0, inplace=True)
episodeagents_df = episodeagents_df.sort_values(by=['EpisodeId'], ascending=True)

episodeagents_df.info(verbose= False)

In [ ]:
def getStats(SUB_ID):

    scores = episodeagents_df[episodeagents_df['SubmissionId']==SUB_ID].UpdatedScore.tolist()
    
    scores_delta = np.diff(scores)
    
    eps = sorted(episodeagents_df[episodeagents_df['SubmissionId']==SUB_ID]['EpisodeId'].values)[2:]
    us = episodeagents_df[(episodeagents_df['EpisodeId'].isin(eps)) & (episodeagents_df['SubmissionId']==SUB_ID)].sort_values('EpisodeId').Reward.values
    them = episodeagents_df[(episodeagents_df['EpisodeId'].isin(eps)) & (episodeagents_df['SubmissionId']!=SUB_ID)].sort_values('EpisodeId').Reward.values
    outcomes = []
    for u,t in zip(us,them):
        if u>t: outcomes.append(1)
        if u<t: outcomes.append(0)
        if u==t: 
            outcomes.append(0.5)
            print(u,t)

    return np.array(scores), np.array(outcomes), np.array(scores_delta)

![Screenshot](https://i.imgur.com/5gLyHfn.png)SUB_ID is the number in the end of link in form https://www.kaggle.com/competitions/kore-2022/leaderboard?dialog=episodes-submission-26116349. It also can be seen from submission's page:

In [ ]:
SUB_ID = 26116349 #Copy this for the submission you need to analyse
scores, outcomes, scores_delta = getStats(SUB_ID)

### Submission Score Variation over Time

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(scores, label='scores')
plt.hlines(np.mean(scores), 0, len(scores), color='tab:orange', label=f'mean score {np.mean(scores):.2f}')
plt.hlines(np.median(scores), 0, len(scores), color='tab:olive', label=f'median score {np.median(scores):.0f}')

plt.scatter(np.argmax(scores), np.max(scores), color='tab:green', label=f'top score {np.max(scores)}')
plt.legend()
plt.show()

### Submission Score Change over Games/Instances

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(scores_delta)

plt.scatter(np.argwhere(scores_delta > 0), scores_delta[scores_delta > 0], c='tab:green', label='Positive')
plt.scatter(np.argwhere(scores_delta < 0), scores_delta[scores_delta < 0], c='tab:red', label='Negative')

plt.hlines(0, 0, len(scores_delta), color='black', linestyles='--')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(outcomes, c='lightgray', linestyle='--')

plt.scatter(np.argwhere(outcomes == 1), outcomes[outcomes == 1], c='tab:green', label='Win')
plt.scatter(np.argwhere(outcomes == 0), outcomes[outcomes == 0], c='tab:red', label='Loss')
plt.scatter(np.argwhere(outcomes == 0.5), outcomes[outcomes == 0.5], c='tab:blue', label='Tie')

plt.hlines(np.mean(outcomes), 0, len(outcomes), color='tab:orange', label='win rate')
plt.legend()
plt.title(f'Current win rate = {np.mean(outcomes):.3f}')
plt.show()

### Win Rate change over Matches

In [ ]:
plt.figure(figsize=(15, 8))

plt.plot(range(1, len(outcomes)+1), [sum(outcomes[:n])/n for n in range(1, len(outcomes)+1)], label='win rate')
plt.hlines(np.mean(outcomes), 1, len(outcomes), color='tab:orange', label='current win rate')
plt.legend()
plt.show()